In [80]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

In [81]:
# Importing The Dataset
train_dataset=pd.read_csv("C:\\Users\\aakri\\OneDrive\\Desktop\\Vaibhav\\tweet-sentiment-extraction\\train.csv")
test_dataset=pd.read_csv("C:\\Users\\aakri\\OneDrive\\Desktop\\Vaibhav\\tweet-sentiment-extraction\\test.csv")


In [86]:
train_dataset.head()


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [90]:
test_dataset.head()

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive


In [91]:
# selecting the last two columns for the dataset
train_dataset=train_dataset.iloc[:,[2,3]]

In [92]:
train_dataset.head()

,selected_text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [12]:
train_dataset[train_dataset['selected_text'].isna()]

,selected_text,sentiment
314,NaN,neutral


In [13]:
train_dataset[train_dataset['sentiment'].isna()]

,selected_text,sentiment


In [93]:
train_dataset.selected_text.iloc[314]='None'

In [94]:
# Null values on test set
test_dataset[test_dataset['text'].isna()]
test_dataset[test_dataset['sentiment'].isna()]

,text,sentiment


In [15]:
train_dataset.dtypes

selected_text    object
sentiment        object
dtype: object

In [95]:
train_dataset.shape

(27481, 2)

In [96]:
test_dataset.shape

(3534, 2)

In [54]:
train_dataset.columns
test_dataset.columns

Index(['text', 'sentiment'], dtype='object')

In [18]:
train_dataset.sentiment.value_counts()

neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [97]:
#Text Cleaning, Librarys on Train set
import re
import nltk                     
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import warnings
warnings.filterwarnings("ignore")
selected_text= re.sub("[^a-zA-z]"," ",train_dataset.selected_text[0])   ## everything non alphabetic from first review is replaced by space

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aakri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [98]:
selected_text=selected_text.lower() ## now convert all the char into lower cases
selected_text= selected_text.split()##convering string to list
selected_text

['i`d', 'have', 'responded', 'if', 'i', 'were', 'going']

In [99]:
ps=PorterStemmer()
selected_text=[ps.stem(word) for word in selected_text if not word in set(stopwords.words('english'))] ## removes word in stopword list and ps.stem includes only root word
selected_text

['i`d', 'respond', 'go']

In [100]:
selected_text=''.join(selected_text)
selected_text

'i`drespondgo'

In [101]:
# doing the cleaning on all the rows, creating a list corpus
corpus=[]

In [102]:
for i in range(0,27481):
    selected_text= re.sub("[^a-zA-z]"," ",train_dataset.selected_text[i])
    selected_text=selected_text.lower()
    selected_text= selected_text.split()
    ps=PorterStemmer()
    selected_text=[ps.stem(word) for word in selected_text if not word in set(stopwords.words('english'))]
    selected_text=' '.join(selected_text)
    corpus.append(selected_text)

In [ ]:
corpus

In [115]:
## Creating The Bag of words model
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=5000)
X_train=cv.fit_transform(corpus).toarray()

In [116]:
print(X_train.shape)
print(y_train.shape)

(27481, 5000)
(27481,)


In [31]:
## too many columns, use max_feature in cvmax
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [105]:
#dependent Variable
y_train=train_dataset.iloc[:,1].values

In [106]:
##TEST SET
corpus_test=[]


In [107]:
# iNDEPENDENT VARIABLE X FOR TEST SET
for i in range(0,3534):
    text= re.sub("[^a-zA-z]"," ",test_dataset.text[i])
    text=text.lower()
    text= text.split()
    ps=PorterStemmer()
    text=[ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    text=' '.join(text)
    corpus_test.append(text)

In [117]:
cv_test= CountVectorizer(max_features=5000)
X_test=cv_test.fit_transform(corpus_test).toarray()

In [118]:
y_test=test_dataset.iloc[:,1].values

In [119]:
print(X_test.shape)
print(y_test.shape)

(3534, 5000)
(3534,)


In [120]:
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)


GaussianNB(priors=None, var_smoothing=1e-09)

In [121]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [122]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[  82  891   28]
 [ 107 1239   84]
 [  57 1016   30]]


In [73]:
print(y_pred)
print(y_test)

['neutral' 'neutral' 'neutral' ... 'neutral' 'negative' 'neutral']
['neutral' 'positive' 'negative' ... 'negative' 'positive' 'positive']


In [123]:
accuracy=(182+1239+30)/(182+1239+30+891+28+107+84+57+1016)
print(accuracy)

0.39928453494771604


In [124]:
# Training with Decision Tree Classification model on the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[420 294 287]
 [621 455 354]
 [423 357 323]]


In [125]:
accuracy=(420+455+323)/(420+455+323+294+287+621+354+423+357)
print(accuracy)

0.3389926428975665
